# DO NOT RUN THIS NOTEBOOK UNTIL THE PREVIOUS NOTEBOOK FINISHES COMPLETELY 

# THE PREVIOUS NOTEBOOK WILL TAKE 20-30 MINUTES.

# PLEASE BE PATIENT

In [1]:
%%bash

### Source the environment
source ~/.bash_profile

AWS_CLUSTER_STATUS=$(aws eks describe-cluster --name ${AWS_CLUSTER_NAME} --region ${AWS_REGION} --query "cluster.status" --output text)

if [ -n "$AWS_CLUSTER_STATUS" ]; then
    if [ $AWS_CLUSTER_STATUS == "ACTIVE" ]; then
        echo "Cluster status:  $AWS_CLUSTER_STATUS.  Please continue."
    else
        echo "Cluster status:  $AWS_CLUSTER_STATUS.  Please wait for status:  Created"
        exit
    fi
else
  echo "Cluster status:  $AWS_CLUSTER_STATUS.  Please wait for status:  Created"
  exit
fi

Cluster status:  ACTIVE.  Please continue.


# DO NOT CONTINUE UNTIL THE CLUSTER STATUS IS `ACTIVE` ^^ ABOVE ^^.

# Associate IAM Policies with EKS Worker Nodes

In [2]:
%%bash

### Source the environment
source ~/.bash_profile
    
AWS_CLUSTER_STATUS=$(aws eks describe-cluster --name ${AWS_CLUSTER_NAME} --region ${AWS_REGION} --query "cluster.status" --output text)

if [ -n "$AWS_CLUSTER_STATUS" ]; then
    if [ $AWS_CLUSTER_STATUS == "ACTIVE" ]; then
        echo "Cluster status:  $AWS_CLUSTER_STATUS."
        aws iam list-roles \
            | jq -r ".Roles[] \
            | select(.RoleName \
            | startswith(\"eksctl-$AWS_CLUSTER_NAME\") and contains(\"NodeInstanceRole\")) \
            .RoleName"

        aws iam list-roles \
            | jq -r ".Roles[] \
            | select(.RoleName \
            | startswith(\"eksctl-$AWS_CLUSTER_NAME\") and contains(\"NodeInstanceRole\")) \
            .Arn" 

        export INSTANCE_ROLE_NAME=$(aws iam list-roles \
            | jq -r ".Roles[] \
            | select(.RoleName \
            | startswith(\"eksctl-$AWS_CLUSTER_NAME\") and contains(\"NodeInstanceRole\")) \
            .RoleName")
        if [ -n "$INSTANCE_ROLE_NAME" ]; then
            echo "INSTANCE_ROLE_NAME:  $INSTANCE_ROLE_NAME"
            echo "export INSTANCE_ROLE_NAME=${INSTANCE_ROLE_NAME}" | tee -a ~/.bash_profile
        else
            echo "Please re-run this notebook after cluster status:  ACTIVE."
        fi

        export INSTANCE_PROFILE_ARN=$(aws iam list-roles \
            | jq -r ".Roles[] \
            | select(.RoleName \
            | startswith(\"eksctl-$AWS_CLUSTER_NAME\") and contains(\"NodeInstanceRole\")) \
            .Arn")
        if [ -n "$INSTANCE_PROFILE_ARN" ]; then
            echo "INSTANCE_PROFILE_ARN:  $INSTANCE_PROFILE_ARN"
            echo "export INSTANCE_PROFILE_ARN=${INSTANCE_PROFILE_ARN}" | tee -a ~/.bash_profile
        else
            echo "Please re-run this notebook after cluster status:  ACTIVE."
        fi
    else
        echo "Cluster status:  $AWS_CLUSTER_STATUS.  Please wait for status:  Created"
        exit
    fi
else
  echo "Cluster status:  $AWS_CLUSTER_STATUS.  Please wait for status:  Created"
  exit
fi

Cluster status:  ACTIVE.
eksctl-demo-nodegroup-ng-7543a4d2-NodeInstanceRole-1451POJ5UFX0F
arn:aws:iam::231218423789:role/eksctl-demo-nodegroup-ng-7543a4d2-NodeInstanceRole-1451POJ5UFX0F
INSTANCE_ROLE_NAME:  eksctl-demo-nodegroup-ng-7543a4d2-NodeInstanceRole-1451POJ5UFX0F
export INSTANCE_ROLE_NAME=eksctl-demo-nodegroup-ng-7543a4d2-NodeInstanceRole-1451POJ5UFX0F
INSTANCE_PROFILE_ARN:  arn:aws:iam::231218423789:role/eksctl-demo-nodegroup-ng-7543a4d2-NodeInstanceRole-1451POJ5UFX0F
export INSTANCE_PROFILE_ARN=arn:aws:iam::231218423789:role/eksctl-demo-nodegroup-ng-7543a4d2-NodeInstanceRole-1451POJ5UFX0F


# Attach Roles
* Allow Access from/to the Elastic Container Registry (ECR)
* This allows our cluster worker nodes to load custom Docker images (ie. models) from ECR. 
* We will load these custom Docker images in a later section.

In [3]:
%%bash

source ~/.bash_profile

if [ -n "$INSTANCE_ROLE_NAME" ]; then
    echo "INSTANCE_ROLE_NAME:  $INSTANCE_ROLE_NAME"
    aws iam attach-role-policy --role-name $INSTANCE_ROLE_NAME --policy-arn arn:aws:iam::aws:policy/AmazonEC2ContainerRegistryFullAccess
    echo "Completed"
else
    echo "Please re-run this notebook when the EKS cluster status is ACTIVE."
    exit
fi

INSTANCE_ROLE_NAME:  eksctl-demo-nodegroup-ng-7543a4d2-NodeInstanceRole-1451POJ5UFX0F
Completed


# Associate IAM and OIDC
To use IAM Roles for Service Accounts in your cluster, you must create an OIDC identity provider in the IAM console.  

See https://docs.aws.amazon.com/eks/latest/userguide/enable-iam-roles-for-service-accounts.html for more info.

# THIS WILL TAKE 5-10 MINUTES.  PLEASE BE PATIENT.

# _If you see `retryable error`s below, this is OK!_

In [4]:
%%bash

### Source the environment
source ~/.bash_profile

AWS_CLUSTER_STATUS=$(aws eks describe-cluster --name ${AWS_CLUSTER_NAME} --region ${AWS_REGION} --query "cluster.status" --output text)

if [ -n "$AWS_CLUSTER_STATUS" ]; then
    if [ $AWS_CLUSTER_STATUS == "ACTIVE" ]; then
        echo "Cluster status:  $AWS_CLUSTER_STATUS."
        eksctl utils associate-iam-oidc-provider --cluster ${AWS_CLUSTER_NAME} --approve
        echo "Completed"
    else
        echo "Cluster status:  $AWS_CLUSTER_STATUS.  Please wait for status:  Created"
        exit
    fi
else
  echo "Cluster status:  $AWS_CLUSTER_STATUS.  Please wait for status:  Created"
  exit
fi

Cluster status:  ACTIVE.
[ℹ]  eksctl version 0.30.0
[ℹ]  using region us-west-2
[!]  retryable error (RequestError: send request failed
caused by: Put "http://169.254.169.254/latest/api/token": context deadline exceeded (Client.Timeout exceeded while awaiting headers)) from ec2metadata/GetToken - will retry after delay of 49.638546ms
[!]  retryable error (RequestError: send request failed
caused by: Put "http://169.254.169.254/latest/api/token": context deadline exceeded (Client.Timeout exceeded while awaiting headers)) from ec2metadata/GetToken - will retry after delay of 115.2587ms
[!]  retryable error (RequestError: send request failed
caused by: Put "http://169.254.169.254/latest/api/token": context deadline exceeded (Client.Timeout exceeded while awaiting headers)) from ec2metadata/GetToken - will retry after delay of 121.65404ms
[!]  retryable error (RequestError: send request failed
caused by: Put "http://169.254.169.254/latest/api/token": context deadline exceeded (Client.Timeo

# _If you see `retryable error`s ^^ above ^^, this is OK!_

In [5]:
%%bash

### Source the environment
source ~/.bash_profile

AWS_OIDC_ISSUER=$(aws eks describe-cluster --name ${AWS_CLUSTER_NAME} --region ${AWS_REGION} --query "cluster.identity.oidc.issuer" --output text)
if [ -n "$AWS_OIDC_ISSUER" ]; then
    echo "AWS_OIDC_ISSUER:  $AWS_OIDC_ISSUER"
    echo "Completed"
else
    echo "AWS_OIDC_ISSUER:  $AWS_OIDC_ISSUER"
    echo "Something is wrong."
fi

AWS_OIDC_ISSUER:  https://oidc.eks.us-west-2.amazonaws.com/id/71AFB612C22B567DAABEB9F13AC00496
Completed


# Update `~/.kube/config` with our new EKS cluster

In [6]:
%%bash

source ~/.bash_profile

aws eks --region ${AWS_REGION} update-kubeconfig --name ${AWS_CLUSTER_NAME} 

Added new context arn:aws:eks:us-west-2:231218423789:cluster/demo to /home/ec2-user/.kube/config


# Verify Your EKS Cluster


In [9]:
%%bash

kubectl get ns


NAME              STATUS   AGE
default           Active   87m
kube-node-lease   Active   87m
kube-public       Active   87m
kube-system       Active   87m


In [10]:
%%bash

kubectl get nodes


NAME                                           STATUS   ROLES    AGE   VERSION
ip-192-168-13-36.us-west-2.compute.internal    Ready    <none>   80m   v1.18.8-eks-7c9bda
ip-192-168-25-72.us-west-2.compute.internal    Ready    <none>   80m   v1.18.8-eks-7c9bda
ip-192-168-34-234.us-west-2.compute.internal   Ready    <none>   80m   v1.18.8-eks-7c9bda
ip-192-168-41-198.us-west-2.compute.internal   Ready    <none>   80m   v1.18.8-eks-7c9bda


In [ ]:
%%javascript
Jupyter.notebook.save_checkpoint();
Jupyter.notebook.session.delete();